In [1]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [2]:
# Данный блок в области 1 НЕ выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
# запрещается скрывать предупреждения системы
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
# pip install

In [148]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

from IPython.display import display

from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, StandardScaler, OneHotEncoder, RobustScaler
from sklearn.preprocessing import LabelEncoder, QuantileTransformer, FunctionTransformer, Normalizer
from sklearn.model_selection import train_test_split, cross_validate, KFold, GridSearchCV, StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsRegressor 

from sklearn.linear_model import ElasticNet, Ridge, Lasso, Lars, LinearRegression
from sklearn.compose import ColumnTransformer, make_column_transformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor

# pip install category-encoders

import category_encoders as ce

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer

from sklearn.metrics import classification_report, mean_absolute_percentage_error

from sklearn.cluster import DBSCAN

from sklearn.svm import SVR

from sklearn.inspection import permutation_importance

### Область работы 2 (поиск  модели .... )

In [118]:
# Данный блок(и) НЕ выполняются преподавателем в области 2
# блок(и) предназначены для поиска лучшей модели 
# должен быть понятен и очевиден отбор параметров модели
# оставляйте свои комментарии и разъяснения
# 
# Запрещается размещать данные блоки за пределами обасти 2
# Все блоки данной области должны быть выполнены
#
# ЗАПРЕЩАЕТСЯ ИСПОЛЬЗОВАТЬ ТЕСТОВЫЙ НАБОР
#
# Путь к тренировочному набору
# 

path_train = 'train_house_B.csv' # содержит только имя файла, без имен папок !!!

In [119]:
df = pd.read_csv(path_train)

In [120]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 408 entries, 0 to 407
Data columns (total 30 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   price   408 non-null    int64  
 1   0       408 non-null    float64
 2   1       408 non-null    float64
 3   2       408 non-null    float64
 4   3       408 non-null    float64
 5   4       408 non-null    float64
 6   5       408 non-null    float64
 7   6       408 non-null    float64
 8   7       408 non-null    float64
 9   8       408 non-null    float64
 10  9       408 non-null    float64
 11  10      408 non-null    float64
 12  11      408 non-null    float64
 13  12      408 non-null    float64
 14  13      408 non-null    float64
 15  14      408 non-null    float64
 16  15      408 non-null    float64
 17  16      408 non-null    float64
 18  17      408 non-null    float64
 19  18      408 non-null    float64
 20  19      408 non-null    float64
 21  20      408 non-null    float64
 22  21

None

In [121]:
df.duplicated().sum()
# df.drop_duplicates(inplace = True)

0

In [122]:
y = np.array(df.price)
X = df.drop(columns=['price'])

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [32]:
start = time.time()

pipe = Pipeline(steps = [
#                             ('preproc',CT),
                            ('classifier',RandomForestRegressor(n_estimators = 1000, random_state=42,
                                                               n_jobs = -1))
                        ])

regressors = [Ridge(), ElasticNet(), Lars(), Lasso(), KNeighborsRegressor(), RandomForestRegressor(),
             LinearRegression(), GradientBoostingRegressor(), SVR()]
param_grid = [
    {
        'classifier': regressors
    },
]

grid = GridSearchCV(pipe, param_grid, cv = KFold(5), return_train_score = True,
                    scoring = 'neg_mean_absolute_percentage_error', n_jobs=-1)
grid.fit(X_train, y_train)

print(f"цикл пройден за {round((time.time() - start)/60, 1)} минут")

цикл пройден за 0.0 минут


In [33]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

,1,7,0,6,3,2,4,5,8
mean_fit_time,0.001497,0.128412,0.002764,0.001571,0.001539,0.003414,0.000898,0.211655,0.003428
std_fit_time,0.000427,0.012659,0.000859,0.000798,0.000371,0.000906,0.000225,0.004885,0.000899
mean_score_time,0.000613,0.001756,0.002266,0.000772,0.000497,0.000829,0.033997,0.00287,0.001046
std_score_time,0.000229,0.00066,0.001546,0.000478,0.000064,0.000316,0.001736,0.000133,0.000043
param_classifier,ElasticNet(),GradientBoostingRegressor(),Ridge(),LinearRegression(),Lasso(),Lars(),KNeighborsRegressor(),RandomForestRegressor(),SVR()
params,{'classifier': ElasticNet()},{'classifier': GradientBoostingRegressor()},{'classifier': Ridge()},{'classifier': LinearRegression()},{'classifier': Lasso()},{'classifier': Lars()},{'classifier': KNeighborsRegressor()},{'classifier': RandomForestRegressor()},{'classifier': SVR()}
split0_test_score,-0.185598,-0.19522,-0.190885,-0.19114,-0.191139,-0.19114,-0.203741,-0.224149,-0.293338
split1_test_score,-0.198957,-0.191059,-0.202641,-0.20274,-0.20274,-0.20274,-0.226579,-0.207146,-0.284024
split2_test_score,-0.152104,-0.143644,-0.148543,-0.148701,-0.1487,-0.148701,-0.183351,-0.147892,-0.29377
split3_test_score,-0.251836,-0.256806,-0.262491,-0.262648,-0.262822,-0.262823,-0.20859,-0.258291,-0.363493


In [70]:
model = ElasticNet()

parameters = {
    'alpha': [0.001, 0.01, 0.1, 1, 10],
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
}

grid_search = GridSearchCV(model, parameters, cv=5)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

best_alpha = best_model.alpha
best_l1_ratio = best_model.l1_ratio

print('Best alpha:', best_alpha)
print('Best l1_ratio:', best_l1_ratio)

Best alpha: 1
Best l1_ratio: 0.7


In [155]:
en = ElasticNet(alpha = 1, l1_ratio = 0.7)
en.fit(X_train, y_train)
y_pred_best = en.predict(X_test)
y_pred_best_train = en.predict(X_train)
print('MAPE на трэйне: ',mean_absolute_percentage_error(y_train, y_pred_best_train))
print('MAPE на тесте: ',mean_absolute_percentage_error(y_test, y_pred_best))

MAPE на трэйне:  0.16944421410400906
MAPE на тесте:  0.16339710507966823


In [125]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [126]:
df.head()

,price,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28
0,2604000,1.222962,0.047686,-1.033389,-2.465344,0.831065,0.802990,1.590989,0.727946,0.893957,-1.503143,0.0,-1.091075,-0.223185,-1.261316,-0.198191,-0.734539,-0.465315,-0.219265,-0.263991,-0.141322,-0.554035,-1.308863,-0.570187,-0.882735,-0.805741,-0.929397,0.232557,-0.679063,1.307431
1,4900000,-0.091662,-0.356325,-0.486840,-2.465344,-0.126718,-0.024866,-0.323972,1.658337,1.751428,-1.573310,0.0,-1.536530,-0.223185,-1.004916,-0.301373,1.361397,2.149083,-0.219265,-0.263991,-0.141322,-0.554035,0.047278,-0.570187,-0.897850,-0.805741,0.224410,0.864016,1.472618,-0.478408
2,4480000,1.222962,-1.064819,0.338749,0.405623,-0.854973,-0.654330,0.525426,0.585866,-1.661101,1.007304,0.0,-0.693173,-0.223185,0.427914,-0.636874,-0.734539,-0.465315,-0.219265,-0.263991,-0.141322,-0.554035,-1.308863,-0.570187,-0.491114,0.355976,-0.929397,1.118967,1.472618,1.307431
3,3234000,1.222962,-1.570692,-0.696696,0.405623,-0.670784,-0.024866,0.557116,-1.280083,1.273040,-1.163932,0.0,1.388846,-0.223185,0.064636,-0.588288,1.361397,-0.465315,-0.219265,-0.263991,-0.141322,-0.554035,1.403419,-0.570187,-0.724485,-0.805741,0.224410,0.924687,-0.679063,-0.478408
4,3780000,-1.406286,-1.563680,-1.058295,0.405623,-0.126718,-0.879174,-1.649281,0.496669,0.255596,0.928381,0.0,0.969821,-0.223185,0.534076,1.710224,-0.734539,-0.465315,-0.219265,-0.263991,-0.141322,1.804941,0.047278,-0.570187,-0.306965,-0.805741,1.378217,0.170486,-0.679063,-0.478408


In [156]:
en = ElasticNet(alpha = 1, l1_ratio = 0.7)
en.fit(X, y)

ElasticNet(alpha=1, l1_ratio=0.7)

In [157]:
threshold = 0.02

In [158]:
result = permutation_importance(en, X, y, n_repeats=10, random_state=42)
feature_importances = result.importances_mean
selected_features = X.columns[feature_importances > threshold]
selected_features

Index(['0', '2', '12', '15', '20', '22', '24', '25', '27'], dtype='object')

In [129]:
def features_selection_en(data):
    selected_features = data.columns[result.importances_mean > threshold]
    df_filtered = data.loc[:, selected_features]
    return df_filtered

In [130]:
features_transformer_en = FunctionTransformer(features_selection_en, validate=False)

features_selection_transformer = Pipeline(steps = [
    ('features_selection_transformer', features_transformer_en),
    ('scaler', StandardScaler())
    ])

In [131]:
col = X.columns

In [132]:
CT = ColumnTransformer([
        ("features_selection_transformer", features_selection_transformer, col),
        ]
    ,remainder='drop'
#     ,remainder='passthrough'
)

display(CT)
CT.set_output(transform ='pandas')
res_ct = CT.fit_transform(X_train)
pd.DataFrame(res_ct).head()

ColumnTransformer(transformers=[('features_selection_transformer',
                                 Pipeline(steps=[('features_selection_transformer',
                                                  FunctionTransformer(func=<function features_selection_en at 0x131dbefc0>)),
                                                 ('scaler', StandardScaler())]),
                                 Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28'],
      dtype='object'))])

,features_selection_transformer__0,features_selection_transformer__2,features_selection_transformer__12,features_selection_transformer__15,features_selection_transformer__20,features_selection_transformer__22,features_selection_transformer__24,features_selection_transformer__25,features_selection_transformer__27
75,1.187166,-0.749805,-0.20314,-0.712313,1.836145,-0.528877,1.527385,-0.975762,-0.68126
110,-0.110237,-0.768447,-0.20314,-0.712313,-0.544619,-0.528877,0.384647,0.181537,-0.68126
113,1.187166,0.872053,-0.20314,1.403878,-0.544619,-0.528877,-0.758091,0.181537,-0.68126
16,-1.407639,-1.131967,-0.20314,-0.712313,-0.544619,-0.528877,-0.758091,-0.975762,-0.68126
66,-0.110237,2.503233,-0.20314,-0.712313,-0.544619,-0.528877,0.384647,-0.975762,-0.68126


In [134]:
best_pipe = Pipeline(steps = [
                            ('preproc',CT),
                            ('estimator',ElasticNet(alpha = 0.1, l1_ratio = 0.7))
                        ])

best_pipe.fit(X_train, y_train)
y_pred_best = best_pipe.predict(X_test)
y_pred_best_train = best_pipe.predict(X_train)
print('MAPE на трэйне: ',mean_absolute_percentage_error(y_train, y_pred_best_train))
print('MAPE на тесте: ',mean_absolute_percentage_error(y_test, y_pred_best))

MAPE на трэйне:  0.17059141147293239
MAPE на тесте:  0.17628229419598404


In [151]:
pipe = Pipeline([('preprocessing', CT),
                 ('regressor', ElasticNet())])

cross_val = KFold(n_splits=7, shuffle=True)

param_grid = [{
    'regressor': [ElasticNet()],
    'regressor__alpha': [0.001, 0.01, 0.1, 1, 10],
    'regressor__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],
    'preprocessing__features_selection_transformer__scaler': [MinMaxScaler(), StandardScaler(),
                                                              RobustScaler(), Normalizer()]
}]

grid = GridSearchCV(pipe, param_grid, n_jobs = -1, cv = cross_val,
                    return_train_score = True, scoring = 'neg_mean_absolute_percentage_error')
grid.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=7, random_state=None, shuffle=True),
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('features_selection_transformer',
                                                                         Pipeline(steps=[('features_selection_transformer',
                                                                                          FunctionTransformer(func=<function features_selection_en at 0x131dbefc0>)),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['0', '1', '2', '3', '...
                                       ('regressor', ElasticNet())]),
             n_jobs=-1,
             param_grid=[{'preprocessing__features_selection_transformer__scaler': [MinMaxScaler(),
                                                                                    StandardScaler(),
                                                                                    RobustScaler(),
                                                                                    Normalizer()],
                          'regressor': [ElasticNet(alpha=0.1, l1_ratio=0.1)],
                          'regressor__alpha': [0.001, 0.01, 0.1, 1, 10],
                          'regressor__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]}],
             return_train_score=True,
             scoring='neg_mean_absolute_percentage_error')

In [152]:
grid_result = pd.DataFrame(grid.cv_results_).sort_values(["rank_test_score",'std_test_score']).T
grid_result

,35,44,36,37,38,63,64,55,56,39,30,57,31,32,58,33,9,0,1,2,62,59,3,50,51,34,25,52,26,4,27,53,28,54,29,8,7,61,6,43,60,69,5,14,42,89,80,81,82,83,88,84,75,76,77,78,79,13,68,41,87,40,86,12,49,67,85,94,11,66,10,19,65,74,48,93,18,73,92,47,17,91,46,72,16,90,45,99,71,15,24,70,98,23,97,22,96,21,95,20
mean_fit_time,0.0017,0.003657,0.001929,0.002083,0.002151,0.004094,0.003384,0.003111,0.003822,0.003185,0.002616,0.00342,0.003104,0.00226,0.003399,0.001934,0.002374,0.003105,0.003021,0.002393,0.003237,0.003896,0.002258,0.003325,0.002576,0.001923,0.002245,0.00221,0.001711,0.003043,0.003906,0.002413,0.002449,0.003184,0.002643,0.001884,0.002727,0.002238,0.002465,0.00374,0.002626,0.003031,0.002559,0.001724,0.001918,0.003172,0.002204,0.002807,0.002164,0.002589,0.002475,0.00263,0.001676,0.002676,0.002392,0.002169,0.002582,0.002038,0.002154,0.002721,0.002315,0.002066,0.002614,0.001892,0.002757,0.003024,0.002492,0.001819,0.002012,0.002717,0.002179,0.002043,0.002794,0.002539,0.002469,0.0019,0.00279,0.003711,0.002162,0.002678,0.002205,0.002687,0.002904,0.003221,0.001949,0.003234,0.002921,0.001833,0.002836,0.002811,0.001956,0.002154,0.001852,0.001735,0.002145,0.002064,0.002103,0.002337,0.001889,0.001813
std_fit_time,0.000078,0.001612,0.000694,0.000614,0.000895,0.002237,0.001666,0.001485,0.001896,0.001426,0.001425,0.001651,0.001653,0.001111,0.001804,0.000299,0.000996,0.000321,0.000626,0.000267,0.001275,0.002156,0.000417,0.001746,0.000712,0.000712,0.00128,0.00021,0.000104,0.000907,0.002762,0.000526,0.000947,0.001691,0.001178,0.000289,0.001363,0.000115,0.000257,0.001087,0.000612,0.001082,0.000879,0.000151,0.000521,0.001312,0.001163,0.00137,0.001177,0.001517,0.001096,0.001439,0.00016,0.001304,0.001075,0.001106,0.001281,0.000732,0.000178,0.001115,0.001017,0.000768,0.001191,0.000202,0.00117,0.001455,0.001217,0.000131,0.000887,0.001527,0.00053,0.000615,0.001572,0.000832,0.001035,0.000255,0.000886,0.001963,0.000535,0.001274,0.00083,0.001317,0.00167,0.001687,0.000307,0.001257,0.001712,0.000139,0.000981,0.00158,0.000606,0.00014,0.000056,0.000214,0.000289,0.000725,0.0003,0.00092,0.000204,0.000298
mean_score_time,0.00133,0.00154,0.000882,0.001204,0.00095,0.001582,0.001824,0.001425,0.001601,0.001186,0.001487,0.001631,0.001147,0.001082,0.001972,0.001292,0.001323,0.001394,0.001302,0.001123,0.001086,0.001222,0.00127,0.001655,0.001073,0.000869,0.001552,0.00088,0.000869,0.001152,0.001407,0.000978,0.000934,0.001014,0.001151,0.000908,0.001082,0.001273,0.0014,0.001285,0.000946,0.001158,0.001464,0.000981,0.000904,0.001578,0.001224,0.001676,0.000972,0.001228,0.000988,0.001291,0.000916,0.001301,0.001717,0.001324,0.001458,0.000937,0.000877,0.000962,0.000987,0.001176,0.001573,0.001351,0.001611,0.001193,0.001349,0.000946,0.00117,0.001106,0.000939,0.001155,0.001165,0.001142,0.001187,0.000959,0.00129,0.001693,0.000971,0.000953,0.001358,0.001007,0.001209,0.001164,0.000979,0.001341,0.001777,0.000963,0.001099,0.001449,0.000944,0.001146,0.000966,0.000899,0.001021,0.000954,0.001275,0.001216,0.000986,0.000893
std_score_time,0.000689,0.000865,0.000065,0.000682,0.000118,0.001155,0.001311,0.000873,0.000761,0.0005,0.000763,0.000845,0.000511,0.000427,0.001621,0.00065,0.000762,0.000144,0.000173,0.000129,0.000374,0.000565,0.00055,0.00087,0.000372,0.000055,0.001087,0.00003,0.000017,0.000276,0.001147,0.0002,0.000061,0.00029,0.000542,0.000055,0.000176,0.00089,0.000503,0.000513,0.000071,0.000563,0.00028,0.00015,0.000056,0.000894,0.000723,0.000931,0.000098,0.000677,0.00007,0.000572,0.000029,0.000522,0.00154,0.000439,0.00067,0.000126,0.00003,0.000078,0.000106,0.000736,0.000697,0.000572,0.001068,0.000382,0.000607,0.000024,0.000709,0.000578,0.000073,0.000551,0.000702,0.0004,0.000663,0.000025,0.000525,0.000927,0.00002,0.000124,0.000623,0.000046,0.000517,0.000568,0.000133,0.000783,0.000992,0.000029,0.000409,0.000808,0.000071,0.000597,0.000012,0.00006,0.000062,0.000145,0.000422,0.000757,0.000077,0.000036
param_preprocessing__features_selection_transformer__scaler,StandardScaler(),StandardScaler(),StandardS

In [153]:
best_pipe = Pipeline(steps = [
                            ('preproc',CT),
                            ('estimator',ElasticNet(alpha = 0.1, l1_ratio = 0.1))
                        ])

best_pipe.fit(X_train, y_train)
y_pred_best = best_pipe.predict(X_test)
y_pred_best_train = best_pipe.predict(X_train)
print('MAPE на трэйне: ',mean_absolute_percentage_error(y_train, y_pred_best_train))
print('MAPE на тесте: ',mean_absolute_percentage_error(y_test, y_pred_best))

MAPE на трэйне:  0.17061172676060377
MAPE на тесте:  0.17599125838139665


# КОНЕЦ